In [26]:
# линейная регрессия, Lasso и Ridge регрессии и методы оптимизации
%%latex
$$ (1) y = \langle w, x \rangle$$
$$ (2) Y = Xw $$
$$ (3) w = (X^{T}X)^{-1}X^{T}Y $$
# (1) - запись линейной регрессии в виде скалярного произведения (с наличием байеса (то есть свободного)  члена w0, к которому мы добавили фиктивный признак, равный единице, чтобы удобно было представить в виде скалярного произведения). (2) запись лин рег в матричном виде, где X - матрица (вектор) признаков размера n на k, w - вектор весов размера k. (3) аналитическое решение для минимализации ошибки MSE для матричной формы

<IPython.core.display.Latex object>

In [41]:
# 1) АНАЛИТИЧЕСКОЕ РЕШЕНИЕ
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


class MyLinearRegression:
    def __init__(self, fit_intercept = True):
        self.fit_intercept = fit_intercept # fit_intercept отвечает за наличие байеса 
    
    def fit(self, X, y):
        n, k = X.shape
        X_train = X
        if self.fit_intercept:
            X_train = np.hstack((X, np.ones((n, 1)))) # если есть байес, то добавляем фиктивную 1 
                                                      # для удобного вида в скалярном произведении
        
        self.w = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y
        
    def predict(self, X):
        n, k = X.shape
        
        if self.fit_intercept:
            X_train = np.hstack((X, np.ones((n, 1))))
            
        y_pred = X_train @ self.w
        return y_pred
    
    def get_weights(self):
        return self.w
    
def linear_expression(x):
    return 5 * x + 6


# недостатки аналитического решения: 1) посчитать inv обратную матрицу долго - куб операций
#                                    2) обратная матрица не всегда есть
# обратной матрицы может не быть, если мы имеем дело с вырожденной матрицей (в ней есть линейно 
# зависимые строки или столбцы). поэтому используется более серьезный метод - Градиентная оптимизация
# (градиентный спуск)


#стандартный (медленный вариант градиента)

class MyGradientLinearRegression(MyLinearRegression):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.w = None # итоговые веса
        
        
    def fit(self, X, y, shag = 0.01,  max_iter = 100):
        n, k = X.shape
        # случайно зададим веса
        if self.w is None:
            self.w = np.random.randn(k + 1 if self.fit_intercept else k)
        
        X_train = np.hstack((X, np.ones((n, 1)))) if self.fit_intercept else X
        
        self.losses = []
        
        for iter_num in range(max_iter):
            y_pred = self.predict(X)
            self.losses.append(mean_squared_error(y_pred, y))
            
            grad = self.calc_gradient(X_train, y)
            
            self.w -= shag * grad
            
        return self
    
    def calc_gradient(self, X, y):
        n, k = X.shape
        grad = (2 / n) * np.dot(X.T, (np.dot(X, self.w) - y))
        
        return grad
    
    def get_losses(self):
        return self.losses
    

# стохастический mini-batch. главное его отличие - это использование подвыборки для шага оптимизации
# (а не всей выборки, как в обычном спуске)

class StochasticMiniBatch(MyGradientLinearRegression):
    def __init__(self, n_sample = 10, **kwargs):
        super().__init__(**kwargs)
        self.w = None # итоговые веса
        self.n_sample = n_sample # кол-во элементов для подвыборки
    
    def calc_gradient(self, X, y):
        n, k = X.shape 
        inds = np.random.choice(np.arange(X.shape[0]), size = self.n_sample, replace = False)
        grad = (2 / n) * (np.dot(X.T, np.dot(X, self.w) - y))
        return grad

regressor = StochasticMiniBatch(fit_intercept=True)

l = regressor.fit(X_train[:, np.newaxis], y_train, max_iter=100).get_losses()

predictions = regressor.predict(X_test[:, np.newaxis])
w = regressor.get_weights()
w

array([4.83186993, 5.0042125 ])

In [27]:
%%latex #формула градиентного спуска в матричной форме:
$$\nabla Q(w) = \frac{2}{l}X^{T}(X_w - y)$$

<IPython.core.display.Latex object>

In [19]:
%%latex 
$$w^{t+1} = w^t - a_t\nabla Q(w^{t})$$ # правило обновления весов в градиентном спуске

<IPython.core.display.Latex object>

In [20]:
%%latex # ФОРМУЛА(2) # проверка для остановки в градиенте
$$w^{t} - w^{t-1} < e$$

<IPython.core.display.Latex object>

In [5]:
%%latex
$$Q(w) = \frac{1}{l}\sum_{i=1}^l(\langle w, x\rangle - y_i)$$

<IPython.core.display.Latex object>

In [ ]:
# линейная регрессия в матричном (векторном виде)
# матричная форма намного быстрее обычного градиента
# всё, что меняется - это формула градиента в матричной форме
class LinearRegressionVectorized(BaseEstimator): 

    def __init__(self, epsilon=1e-4, w0 = None, max_steps = 1000, alpha=1e-2):
        self.epsilon = epsilon
        self.w0 = w0
        self.max_steps = max_steps
        self.alpha = alpha
        self.w = None
        self.w_history = []
    def fit(self, X, y):
        l, d = X.shape
        if self.w0 == None:
            self.w0 = np.zeros(d)
        self.w = w0
        for step in range(self.max_steps):
            self.w_history.append(self.w)
            w_new = self.w - self.alpha * self.calc_gradient(X, y)
            if np.linalg.norm(self.w_new - self.w) < epsilon:
                break
        self.w = self.w_new
        return self
    
    def calc_gradient_vectorized(self, X, y):
        l, d = X.shape
        return (2/l)*np.dot(X.T, (np.dot(X, self.w) - y))
        
    def predict(self, X):
        if self.w == None:
            raise Exception 'Модель не обучена'
        return np.dot(X, self.w)